El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD
La tabla de clientes es

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 |                                                  |
| nombre cliente o razón social           |    NOMBRES, PRIMER_APELLIDO, SEGUNDO_APELLIDO, RAZON_SOCIAL, NOMBRE_COMERCIAL |
| número cliente                          |    codigo_cliente                                         |
| estatus                                 |      Por movimientos                             |    
| número contrato                         | NUMERO DE CONTRADO ORIGEN: PR_CONTRATOS          |
| género                                  |     SEXO IF PF ELSE N/A |
| fecha_nacimiento o de constitución      |   FECHA_DE_NACIMIENTO                            |
| entidad federativa nacimiento           |                                   |
| país de nacimiento                      |           PAIS_NACIMIENTO                                       |
| nacionalidad                            |    SE OBTIENE DE MG_PAISES|
| profesión                               |    CODIGO_PROFESION                                              |
| calle                                   |                                 |
| número_exterior                         |                                                  |
| número_interior                         |                                                  |
| colonia_urbanización                    |                                                  |
| delegación_municipio                    |          CODIGO_MUNICIPIO                                        |
| ciudad_población                        |                                                  |
| entidad_federativa                      |          CODIGO_DEPARTAMENTO                          |
| código postal                           |                                                  |
| país                                    |       CODIGO_PAIS                         |
| teléfono                                |                                    |
| correo_electrónico                      |                                                  |
| rfc                                     | NUMERO_IDENTIFICACION     |
| curp                                    |       CURP                                           |
| fiel                                    |          FIEL                                        |
| representante legal                     |    REPRESENTANTE_LEGAL                                              |
| tipo de persona                         |   CODIGO_TIPO_IDENTIFICACION       5:PF 7:PM   |
| clasificación por grado de riesgo       |                                                  |
| pep (persona políticamente expuesta)    |                                                  |
| razón                                   |                                                  |
| producto contratado                     |                   |
| cambios en perfil transaccional         |          INFORMACION NO ENCONTRADA                                        |

**Criterio para Determinar Cliente Activo**

1. **Contratos Activos**:
   - Se consideran los contratos cuya fecha de vencimiento es mayor al día de hoy y cuyo estado de contrato es 1.
   - Al filtrar por fecha de vencimiento, los valores resultantes del estado de contrato son 1 (Activo) o 9 (Cancelado). Se asume que estos valores corresponden al catálogo de estados de préstamo (PR_ESTADOS_PRESTAMO), donde 1 indica "Activo" y 9 "Cancelado".

2. **Filtrado Adicional**:
   - Se realiza una unión con la tabla PR_PRESTAMO mediante el código de cliente y el número de contrato.
   - De esta unión, se identifican clientes con posible no actividad, considerando los siguientes valores del estado del préstamo:
     - 1: Activo
     - 2: Liquidado
     - 6: Castigado
     - 10: Reinstalado
   - Se consideran como clientes activos aquellos con estado 1 (Activo) y 10 (Reinstalado).


In [2]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
from rapidfuzz import process, fuzz
import numpy as np

In [5]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/clientes_activos/24_02_2025/Clientes_activos_SIRAC.dat")

In [8]:
df.columns

Index(['CODIGO_CLIENTE', 'CODIGO_TIPO_IDENTIFICACION', 'NOMBRES',
       'PRIMER_APELLIDO', 'SEGUNDO_APELLIDO', 'SEXO', 'FECHA_DE_NACIMIENTO',
       'RAZON_SOCIAL', 'NOMBRE_COMERCIAL', 'NO_EMPLEADOS', 'VENTAS_NETAS',
       'PAIS', 'NACIONALIDAD', 'DELEGACION_MUNICIPIO', 'ENTIDAD_FEDERATIVA',
       'TIPO_SECTOR', 'TIPO_SUBSECTOR', 'TIPO_RAMA', 'TIPO_CLASE',
       'NUMERO_CONTRATO', 'NUMERO_PRESTAMO', 'FECHA_APERTURA', 'AÑO_REGISTRO',
       'MES_REGISTRO', 'DIA_REGISTRO', 'FECHA_VENCIMIENTO', 'AÑO_VENCIMIENTO',
       'MES_VENCIMIENTO', 'DIA_VENCIMIENTO', 'PRODUCTO',
       'DESC_LINEA_FINANCIERA', 'CODIGO_MONEDA', 'MONTO_APROBADO',
       'MONTO_DISPONIBLE', 'MONTO_INICIAL', 'TIPO_TASA', 'TASA_TOTAL', 'PLAZO',
       'CODIGO_TIPO_AMORTIZACION', 'AMORTIZACION', 'NUMERO_CUOTAS',
       'VALOR_CUOTA', 'CANTIDAD_CUOTAS_PAGADAS'],
      dtype='object')

In [7]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/productos/SIRAC/data/26_02_2025/productos_clientes_SIRAC.dat")

In [9]:
df["GENERO"].value_counts()

GENERO
NO_APLICA    4396
MASCULINO    2843
FEMENINO     1834
Name: count, dtype: int64

In [8]:
map_genero = {"F": "FEMENINO", "M": "MASCULINO", "N/A": "NO APLICA"}
df["GENERO"] = df["GENERO"].apply(lambda x: map_genero.get(x,"NO_APLICA"))

In [3]:
df["NOMBRE_O_RAZON_SOCIAL"] = df.apply(lambda row:(
                                                    row["RAZON_SOCIAL"] if row["TIPO_PERSONA"] == 7
                                                    else f'{row["NOMBRES"]} {row["PRIMER_APELLIDO"]} {row["SEGUNDO_APELLIDO"]}'
                                                    ),axis=1)

In [4]:
df["RFC"] = df["NUMERO_IDENTIFICACION"]

In [5]:
df["RFC"] = df["RFC"].apply(lambda x: normalizar_RFC(x) if pd.notna(x) and verificar_rfc(normalizar_RFC(x)) else "XXXXXXXXXXXXX")

In [6]:
df["TIPO_PERSONA"] =df["TIPO_PERSONA"].apply(lambda x: "PM" if x == 7 else "PF")

In [7]:
df['FECHA_DE_ACTUALIZACION'] = datetime.now().strftime('%d/%m/%y')
col_final = ['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
   'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
   'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
   'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
   'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
   'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
   'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
   'TIPO_PERSONA', 'PRODUCTO_CONTRATADO',
   'SISTEMA_ORIGEN', 'FECHA_DE_ACTUALIZACION']
df = df[col_final]

In [8]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)

In [10]:
if not clt_repetidos.empty:
    df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)
    df = df.drop(drops_ids)
    df = pd.concat([df, df_resul], ignore_index=True)
    df = df.sort_values("NOMBRE_O_RAZON_SOCIAL")

/home/ale1726/proyects/datalake/clientes/clientes_eda/utils/clt_repetidos.py:183: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resul = df_resul.replace("-", np.nan)


In [15]:
name_archivo = os.path.join("/home/ale1726/proyects/datalake/clientes/data/ETL","CLientesT_SIRAC.csv")
df.to_csv(name_archivo,index=False)

In [14]:
df[df["FECHA_NAC_O_CONST"]< "1950-01-15 00:00:00"]

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
2761,NaN,ALEJANDRO MONTERRUBIO RODRIGUEZ,28337042,VIGENTE,40701032,M,1945-09-24 00:00:00,NaN,24.0,MEXICANA,...,NaN,NaN,MORA450924IH5,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
973,NaN,ALEJANDRO TORRES PRIETO,10111710,VIGENTE,40683101,M,1938-11-09 00:00:00,NaN,24.0,MEXICANA,...,NaN,NaN,TOPA381109BB5,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
717,NaN,ALFREDO POSADAS HERNANDEZ,5939177,VIGENTE,40382747,M,1945-08-18 00:00:00,NaN,NaN,MEXICANA,...,NaN,NaN,POHA4508182G9,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
3352,NaN,ANDRES ESCUDERO AGUILAR,28497423,VIGENTE,40602857,M,1945-11-30 00:00:00,NaN,24.0,MEXICANA,...,NaN,NaN,EUAA451130PM0,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
3201,NaN,ANTONIO KURI ZERMENO,28465645,VIGENTE,40412627,M,1943-09-14 00:00:00,NaN,24.0,MEXICANA,...,NaN,NaN,KUZA430914PS9,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
3027,NaN,ARTURO RITO MIJANGOS PEREZ,28421307,VIGENTE,40332848,M,1949-05-22 00:00:00,NaN,24.0,MEXICANA,...,NaN,NaN,MIPA490522G24,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
166,NaN,ARTURO SAAVEDRA AVALOS,3698548,VIGENTE,40497657,M,1949-04-29 00:00:00,NaN,24.0,MEXICANA,...,NaN,NaN,SAAA490429EU1,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
97,NaN,AURELIO CHAVEZ CORONA,1988121,VIGENTE,40435350,M,1948-10-07 00:00:00,NaN,NaN,MEXICANA,...,NaN,NaN,CACX481007RH0,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
1048,NaN,BALTAZAR GUERRERO MEDRANO,10260247,VIGENTE,40705752,M,1938-01-06 00:00:00,NaN,NaN,MEXICANA,...,NaN,NaN,GUMB380106J79,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
1740,NaN,BENITO LOPEZ LOPEZ,23051369,VIGENTE,40609580,M,1947-04-27 00:00:00,NaN,24.0,MEXICANA,...,NaN,NaN,LOLB470427LF5,NaN,NaN,NaN,PF,NaN,SIRAC,10/12/24
